Ce Notebook à été exporter du notebook de kaggle. Pour avoir les output des cells, veuillez importer ce notebook dans un notebook créer sur kaggle.

In [ ]:
import matplotlib.pyplot as plt
import pandas as pd
import numpy as np
import tensorflow as tf
import seaborn as sns

train = pd.read_csv('../input/digit-recognizer/train.csv')
test = pd.read_csv('/kaggle/input/digit-recognizer/test.csv')

In [ ]:
print(train.shape)
train.head(2)

In [ ]:
print(test.shape)
test.head(2)

In [ ]:
x_train = train.iloc[:,1:]
y_train = train.iloc[:,0]

In [ ]:
def to_matrice(l):
    
    """
    @ params : l -> liste contenant les valeurs des pixels
    @ return : m -> matrice de taille 28*28
     
    @ descripton : Transforme la liste applatie en matrice
    """
    
    m = np.empty((28,28))
    
    for i in range(len(m)):
        for j in range(len(m[i])):
            m[i,j] = l[28*i+j]
    
    return m

In [ ]:
n = 5

fig, axs = plt.subplots(1,n)
plt.title('none normalized',loc='center')
for i in range(n):
    m = to_matrice(x_train.iloc[i,:])
    axs[i].imshow(m, cmap=plt.cm.binary)
    axs[i].set_title(str(y_train.iloc[i]))
plt.show


In [ ]:
sns.countplot(y_train)
plt.title("Nombre d'image pour chaque chiffre")
plt.xlabel("Chiffre")
plt.ylabel('Nombre')

In [ ]:
# Normalization des pixels
x_train = tf.keras.utils.normalize(x_train)
test = tf.keras.utils.normalize(test)

# transformation des classes en vecteur 
y_train = tf.keras.utils.to_categorical(y=y_train,
                                       num_classes= y_train.nunique(),
                                       dtype='float64')
y_train[0]

In [ ]:
model = tf.keras.Sequential()

# Flattern permet de transformer les nD array en 1D array et de rajouter un channel si il n'y en a pas
model.add(tf.keras.layers.Flatten())
# 1ere couche
model.add(tf.keras.layers.Dense(392, activation=tf.nn.relu))
# 2eme couche
model.add(tf.keras.layers.Dense(392, activation=tf.nn.relu))
# couche de sortie
model.add(tf.keras.layers.Dense(10, activation=tf.nn.softmax))


model.compile(optimizer ='adam',
              loss ='categorical_crossentropy',
              metrics=['accuracy']
             )

In [ ]:
history = model.fit(x_train.values,y_train,epochs=50,validation_split=0.30,verbose=1)

In [ ]:

fig, ax = plt.subplots(2,1)
# 1ere subplot
ax[0].plot(history.history['loss'],'r',label='loss')
ax[0].plot(history.history['val_loss'],'g',label='val_loss')
ax[0].set_title('NN results')
ax[0].legend()

# 2eme subplot
ax[1].plot(history.history['accuracy'],'b',label='acc')
ax[1].plot(history.history['val_accuracy'],'g',label='val_acc')
#ax[1].set_title('acc')
plt.legend()



In [ ]:
predic = model.predict(test)

In [ ]:
n = 10

# affichage des chiffres predit
l = []
for i in range(10):
    l.append(predic[i].argmax())

print(l)

# affichage des chiffres attendu
fig, axs = plt.subplots(1,n)
#plt.title('none normalized',loc='center')
for i in range(n):
    m = to_matrice(test.iloc[i,:])
    axs[i].imshow(m, cmap=plt.cm.binary)
    #axs[i].set_title(str(i))
    axs[i].axis('off')
plt.show

# CNN

In [ ]:
# Pour utiliser CNN il faut que les images ne soit pas aplati. Pour cela on va les réajuster nos données.
CNN_train = np.array(x_train)
CNN_train = CNN_train.reshape(-1,28,28,1)
CNN_train.shape

In [ ]:
CNN_model = tf.keras.Sequential()

# 1ere couche de convolution avec une fenetre de 4*4 
CNN_model.add(tf.keras.layers.Conv2D( filters=16,input_shape=(28,28,1), kernel_size=(4,4), activation='relu'))

# normalize les inputs.Plus utile vu que les images sont déjà normalizer
#CNN_model.add(tf.keras.layers.BatchNormalization())

# Couche permettant prendre la valeur maximal des pixels dans la fenetre 2*2
# Cela permet de réduire la taille de l'image
CNN_model.add(tf.keras.layers.MaxPool2D(strides=(2,2)))

# 2eme couche de convolution avec une fenetre de 4*4
CNN_model.add(tf.keras.layers.Conv2D( filters=16,input_shape=(28,28,1), kernel_size=(4,4), activation='relu'))

#CNN_model.add(tf.keras.layers.BatchNormalization())

# idem
CNN_model.add(tf.keras.layers.MaxPool2D(strides=(2,2)))

# Dropout permet de mettre à zero certaines valeur du input à une fréquence de 0.25*
# Utile de le faire pour éviter le overfitting 
CNN_model.add(tf.keras.layers.Dropout(0.25))

#flattern permet d'aplatir l'image c'esta dire passer de ND à 1D
CNN_model.add(tf.keras.layers.Flatten())

# 1ere couche
CNN_model.add(tf.keras.layers.Dense(512, activation='relu'))
CNN_model.add(tf.keras.layers.Dropout(0.25))

# 2eme couche
CNN_model.add(tf.keras.layers.Dense(1024, activation='relu'))
CNN_model.add(tf.keras.layers.Dropout(0.5))

# couche de sorti (output layer)
CNN_model.add(tf.keras.layers.Dense(10, activation='softmax'))



CNN_model.compile(optimizer='adam',
                 loss='categorical_crossentropy',
                 metrics=['accuracy'])

In [ ]:
cnn_history = CNN_model.fit(CNN_train,y_train,epochs=50,validation_split=0.3)

In [ ]:
fig, ax = plt.subplots(2,1,sharex=True)
#plt.title()
# 1ere subplot
ax[0].plot(cnn_history.history['loss'],'r',label='loss')
ax[0].plot(cnn_history.history['val_loss'],'g',label='val_loss')
ax[0].set_title('CNN results')
ax[0].legend()

# 2eme subplot
ax[1].plot(cnn_history.history['accuracy'],'b',label='acc')
ax[1].plot(cnn_history.history['val_accuracy'],'g',label='val_acc')
#ax[1].set_title('acc')
ax[1].legend()
plt.xlabel('Epochs')


In [ ]:
print('acc mean : '+str(np.mean(cnn_history.history['val_accuracy'])*100))
print('acc max : '+str(np.max(cnn_history.history['val_accuracy'])*100))
print('loss mean : '+str(np.mean(cnn_history.history['val_loss'])))
print('loss min : '+str(np.min(cnn_history.history['val_loss'])))

In [ ]:
plt.subplot(121)
sns.boxplot(cnn_history.history['val_accuracy'],orient='v')
plt.title('val_acc')
plt.subplot(122)
sns.boxplot(cnn_history.history['val_loss'],orient='v')
plt.title('val_loss')

In [ ]:
# avec le dataset Test

CNN_test = np.array(test)
CNN_test = CNN_test.reshape(-1,28,28,1)
CNN_test.shape

In [ ]:
predict = CNN_model.predict(CNN_test)

In [ ]:
n = 10

# affichage des chiffres predit
l = []
for i in range(10):
    l.append(predict[2500+i].argmax())

print(l)

# affichage des chiffres attendu
fig, axs = plt.subplots(1,n)
#plt.title('none normalized',loc='center')
for i in range(n):
    m = to_matrice(test.iloc[2500+i,:])
    axs[i].imshow(m, cmap=plt.cm.binary)
    #axs[i].set_title(str(i))
    axs[i].axis('off')
plt.show